In [1]:
from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout

pdb = app.PDBFile('/home/sanderroet/scripts/test_show/vis-test.pdb')
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml', 'gtp.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True, 
    ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [2]:
%%time
print('Minimizing...')
simulation.minimizeEnergy()
print('Done minimizing')

Minimizing...
Done minimizing
CPU times: user 5h 25min, sys: 30.4 s, total: 5h 25min 31s
Wall time: 1h 23min 11s


In [3]:
%%time
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.reporters.append(app.DCDReporter('trajectory_eq.dcd', 10))
simulation.reporters.append(app.PDBReporter('trajectory_eq.pdb', 10))
print('Equilibrating...')
simulation.step(100)



Equilibrating...
CPU times: user 9min 40s, sys: 1.26 s, total: 9min 42s
Wall time: 2min 33s


In [4]:
%%time
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 10))
simulation.reporters.append(app.PDBReporter('trajectory.pdb', 10))
simulation.reporters.append(app.StateDataReporter(stdout, 10, step=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
    speed=True, totalSteps=1000, separator='\t'))

print('Running Production...')
simulation.step(1000)
print('Done!')

Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
11.0%	110	-348621.34386	171.366652177	0	--
12.0%	120	-348335.903568	174.639306576	0.0962	26:20
13.0%	130	-348160.775765	178.923903496	0.0945	26:31
14.0%	140	-347795.911313	181.852029521	0.0951	26:01
15.0%	150	-347351.385001	184.010812253	0.0954	25:39
16.0%	160	-346737.090937	185.893979283	0.0948	25:30
17.0%	170	-346624.695078	190.232111691	0.0951	25:07
18.0%	180	-345852.675409	190.621115521	0.0953	24:47
19.0%	190	-345668.905963	194.027541386	0.0949	24:34
20.0%	200	-345491.926173	197.594542951	0.0951	24:13
21.0%	210	-344857.961674	198.346103712	0.0952	23:53
22.0%	220	-344395.727812	199.162252384	0.0954	23:33
23.0%	230	-343877.794912	200.645196377	0.0951	23:18
24.0%	240	-343500.939111	202.307527125	0.0952	22:59
25.0%	250	-343153.588496	204.410632021	0.0953	22:39
26.0%	260	-342851.82005	206.747265969	0.0954	22:21
27.0%	270	-342321.060178	207.633774995	0.0951	22:06
